In [1]:
import pandas as pd
import numpy as np

### Cargar datos

In [2]:
listings = pd.read_csv("Beijing Airbnb Data(clean)/listings_clean.csv")

In [49]:
calendar = pd.read_csv("Beijing Airbnb Data(clean)/calendar_detail.csv")

In [16]:
reviews = pd.read_csv("Beijing Airbnb Data(clean)/reviews_detail.csv")

In [5]:
print(listings.shape,reviews.shape,calendar.shape)

(28448, 15) (202099, 5) (10384980, 7)


#### Pre-procesamiento listings

In [6]:
listings.head(5)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,朝阳区,39.89503,116.45163,Entire home/apt,792,1,89,2019-03-04,0.85,9,341
1,100213,The Great Wall Box Deluxe Suite A团园长城小院东院套房,527062,Joe,密云县,40.68434,117.17231,Private room,1201,1,2,2017-10-08,0.10,4,0
2,128496,Heart of Beijing: House with View 2,467520,Cindy,东城区,39.93213,116.42200,Entire home/apt,389,3,259,2019-02-05,2.70,1,93
3,161902,cozy studio in center of Beijing,707535,Robert,东城区,39.93357,116.43577,Entire home/apt,376,1,26,2016-12-03,0.28,5,290
4,162144,"nice studio near subway, sleep 4",707535,Robert,朝阳区,39.93668,116.43798,Entire home/apt,537,1,37,2018-08-01,0.40,5,352


In [18]:
listings.isna().sum()

id                  0
name                0
host_id             0
host_name           0
room_type           0
price               0
minimum_nights      0
availability_365    0
dtype: int64

In [11]:
listings['last_review']=listings['last_review'].replace(np.nan, 'No reviews', regex=True)

In [12]:
listings.isna().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [13]:
loc=listings[['id','latitude','longitude']]

In [15]:
loc.to_csv('db_final/location.csv',index=False)

In [16]:
listings=listings.drop(['neighbourhood','number_of_reviews','last_review','reviews_per_month',
                        'calculated_host_listings_count','latitude','longitude'], axis=1)

In [17]:
listings.to_csv('listings_final.csv',index=False) #para traducir

In [5]:
listings = pd.read_csv("table_listing.csv") #traducido

In [6]:
listings.head(5)

,id,name,host_id,host_name,room_type,price,minimum_nights,availability_365,Unnamed: 8,name_en,host_name_en
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,Entire home/apt,792,1,341,NaN,Modern and Comfortable Living in CBD,East Apartments
1,100213,The Great Wall Box Deluxe Suite A团园长城小院东院套房,527062,Joe,Private room,1201,1,0,NaN,The Great Wall Box Deluxe Suite A Group Garden...,Joe
2,128496,Heart of Beijing: House with View 2,467520,Cindy,Entire home/apt,389,3,93,NaN,Heart of Beijing: House with View 2,Cindy
3,161902,cozy studio in center of Beijing,707535,Robert,Entire home/apt,376,1,290,NaN,cozy studio in center of Beijing,Robert
4,162144,"nice studio near subway, sleep 4",707535,Robert,Entire home/apt,537,1,352,NaN,"nice studio near subway, sleep 4",Robert


In [7]:
listings=listings.drop(['name','host_name', 'Unnamed: 8'], axis=1)
listings.rename(columns={'name_en': 'name', 'host_name_en':'host_name'}, inplace=True)

In [8]:
listings=listings[['id','name','host_id','host_name','room_type','price','minimum_nights','availability_365']]

In [9]:
listings['name']=listings['name'].str.replace(r'\n', ' ')
listings['name']=listings['name'].str.replace('\s+', ' ')
listings['name']=listings['name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')

<ipython-input-9-d2caa83fa581>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace(r'\n', ' ')
<ipython-input-9-d2caa83fa581>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace('\s+', ' ')
<ipython-input-9-d2caa83fa581>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  listings['name']=listings['name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')


In [10]:
listings.head(5)

,id,name,host_id,host_name,room_type,price,minimum_nights,availability_365
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,Entire home/apt,792,1,341
1,100213,The Great Wall Box Deluxe Suite A Group Garden...,527062,Joe,Private room,1201,1,0
2,128496,Heart of Beijing House with View 2,467520,Cindy,Entire home/apt,389,3,93
3,161902,cozy studio in center of Beijing,707535,Robert,Entire home/apt,376,1,290
4,162144,"nice studio near subway, sleep 4",707535,Robert,Entire home/apt,537,1,352


In [11]:
listings.to_csv('db_final/listings.csv',index=False)

#### Pre-procesamiento reviews

In [17]:
for x in reviews.listing_id.array:
    if x not in listings.id.array:
        reviews.drop(reviews.loc[reviews['listing_id']==x].index, inplace=True)

In [18]:
reviews.shape

(202018, 5)

In [19]:
len(reviews.listing_id.unique())

17293

In [37]:
n = reviews.shape[0]
reviews_sample = reviews.groupby('listing_id', group_keys=False).apply(lambda x: x.sample(int(np.ceil(50000*len(x)/len(reviews))))).sample(frac=1).reset_index(drop=True)



In [38]:
len(reviews.listing_id.unique())

17293

In [39]:
reviews_sample.head(5)

,listing_id,id,date,reviewer_name,comments
0,13812306,93210846,11-08-16,代,【我宅】位置便利 大门门面不大 晚上入住的住客要注意看一下 距离周围景点都很近 吃的喝的一条...
1,28789220,404695224,25-01-19,Congge,"非常感谢小姐姐给我们这一个月的住宿,一切来说都很完美,非常好沟通,住的十分舒服,距离上班和地..."
2,25020050,263824879,13-05-18,Xeen,"很好的房东,很干净的房间~"
3,29947072,426745032,22-03-19,记忆,"房间很干净,沟通也很及时,位置好找,交通非常便利,环境很安全,性价比高｡"
4,32509266,426722613,22-03-19,忠正,"房东是个帅气的小伙子,非常安静的屋子,也干净,总之是非常愉快的体验!"


In [40]:
reviews_sample.shape

(58381, 5)

In [41]:
reviews_sample.to_csv('reviews_final.csv',index=False) #a traducir

In [2]:
reviews = pd.read_csv("reviews_final2.csv") #traducido

In [3]:
reviews.head(5)

,listing_id,id,date,reviewer_name,comments,rev_new,com_new
0,13812306.0,93210846.0,11-08-16,代,【我宅】位置便利 大门门面不大 晚上入住的住客要注意看一下 距离周围景点都很近 吃的喝的一条街都有,generation,[My House] The position of the convenience of ...
1,28789220.0,404695224.0,25-01-19,Congge,"非常感谢小姐姐给我们这一个月的住宿,一切来说都很完美,非常好沟通,住的十分舒服,距离上班和地...",Congge,I am very grateful to the little sister to giv...
2,25020050.0,263824879.0,13-05-18,Xeen,"很好的房东,很干净的房间~",Xeen,"Very good landlord, very clean room ~"
3,29947072.0,426745032.0,22-03-19,记忆,"房间很干净,沟通也很及时,位置好找,交通非常便利,环境很安全,性价比高｡",memory,"The room is very clean, communication is also ..."
4,32509266.0,426722613.0,22-03-19,忠正,"房东是个帅气的小伙子,非常安静的屋子,也干净,总之是非常愉快的体验!",Loyal,"The landlord is a handsome young man, very qui..."


In [5]:
reviews=reviews.drop(['reviewer_name','comments'], axis=1)
reviews.rename(columns={'rev_new': 'reviewer_name', 'com_new':'comments'}, inplace=True)

In [6]:
reviews.head(5)

,listing_id,id,date,reviewer_name,comments
0,13812306.0,93210846.0,11-08-16,generation,[My House] The position of the convenience of ...
1,28789220.0,404695224.0,25-01-19,Congge,I am very grateful to the little sister to giv...
2,25020050.0,263824879.0,13-05-18,Xeen,"Very good landlord, very clean room ~"
3,29947072.0,426745032.0,22-03-19,memory,"The room is very clean, communication is also ..."
4,32509266.0,426722613.0,22-03-19,Loyal,"The landlord is a handsome young man, very qui..."


In [7]:
reviews['comments']=reviews['comments'].str.replace(r'\n', ' ')
reviews['comments']=reviews['comments'].str.replace('\s+', ' ')
reviews['comments']=reviews['comments'].str.replace('[^0-9a-zA-Z.,\s]', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace(r'\n', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace('\s+', ' ')
reviews['reviewer_name']=reviews['reviewer_name'].str.replace('[^0-9a-zA-Z.,\s]', ' ')

<ipython-input-7-4f1c0b100152>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace(r'\n', ' ')
<ipython-input-7-4f1c0b100152>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace('\s+', ' ')
<ipython-input-7-4f1c0b100152>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['comments']=reviews['comments'].str.replace('[^0-9a-zA-Z.,\s]', ' ')
<ipython-input-7-4f1c0b100152>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewer_name']=reviews['reviewer_name'].str.replace(r'\n', ' ')
<ipython-input-7-4f1c0b100152>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewer_name']=reviews['reviewer_name'].str.replace('\s+', ' 

In [8]:
reviews.head(5)

,listing_id,id,date,reviewer_name,comments
0,13812306.0,93210846.0,11-08-16,generation,My House The position of the convenience of ...
1,28789220.0,404695224.0,25-01-19,Congge,I am very grateful to the little sister to giv...
2,25020050.0,263824879.0,13-05-18,Xeen,"Very good landlord, very clean room"
3,29947072.0,426745032.0,22-03-19,memory,"The room is very clean, communication is also ..."
4,32509266.0,426722613.0,22-03-19,Loyal,"The landlord is a handsome young man, very qui..."


In [9]:
reviews.isna().sum()

listing_id        2
id                2
date              2
reviewer_name    26
comments          1
dtype: int64

In [10]:
reviews.dropna(inplace=True)

In [11]:
reviews.isna().sum()

listing_id       0
id               0
date             0
reviewer_name    0
comments         0
dtype: int64

In [12]:
reviews.shape

(58356, 5)

In [19]:
reviews = reviews.astype({"listing_id": int, "id": int})
reviews['date'] = pd.to_datetime(reviews['date'], format='%d-%m-%y').dt.strftime('%Y-%m-%d')


In [20]:
reviews.head(10)

,listing_id,id,date,reviewer_name,comments
0,13812306,93210846,2016-08-11,generation,My House The position of the convenience of ...
1,28789220,404695224,2019-01-25,Congge,I am very grateful to the little sister to giv...
2,25020050,263824879,2018-05-13,Xeen,"Very good landlord, very clean room"
3,29947072,426745032,2019-03-22,memory,"The room is very clean, communication is also ..."
4,32509266,426722613,2019-03-22,Loyal,"The landlord is a handsome young man, very qui..."
5,12642399,264523265,2018-05-14,Handsome,"Very good. Staying up is very comfortable, the..."
6,24721135,429429664,2019-03-28,Jiayi,"The homeowner is very enthusiastic, the house ..."
7,32407931,437694932,2019-04-15,Phort,"Quiet and comfortable, convenient transportation."
8,15889537,126873223,2017-01-14,Jade,"The room is very big, there should be, live ve..."
9,21488749,292831177,2018-07-19,Shang Ze,"The passionate, neat, complete facilities, a g..."


In [16]:
reviews.to_csv('db_final/reviews.csv',index=False)

#### Pre-procesamiento calendar

In [51]:
calendar.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1165040,2019-04-17,f,$511.00,$511.00,1.0,1125.0
1,1165040,2019-04-18,t,$511.00,$511.00,1.0,1125.0
2,1165040,2019-04-19,t,$511.00,$511.00,1.0,1125.0
3,1165040,2019-04-20,t,$511.00,$511.00,1.0,1125.0
4,1165040,2019-04-21,t,$511.00,$511.00,1.0,1125.0


In [52]:
len(calendar.listing_id.unique())

28452

In [54]:
sample_calendar = calendar.groupby('listing_id', group_keys=False).apply(lambda x: x.sample(int(np.ceil(50000*len(x)/len(calendar))))).sample(frac=1).reset_index(drop=True)
sample_calendar['price']=sample_calendar['price'].str.replace(r',', '')
sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r',', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r'$', '')

<ipython-input-54-f72a22eee1d9>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')
<ipython-input-54-f72a22eee1d9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r'$', '')


In [55]:
len(sample_calendar.listing_id.unique())

28452

In [56]:
sample_calendar.reset_index(drop=True, inplace=True)
sample_calendar.dropna(inplace=True)

In [57]:
sample_calendar.shape

(56902, 7)

In [58]:
sample_calendar.to_csv('db_final/calendar.csv',index=False)